In [40]:
import torch
import numpy as np

In [41]:
data = [[1, 2], [3, 4]]

In [42]:
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

In [43]:
x_data.size()

torch.Size([2, 2])

In [44]:
npm_array = np.array(data)
npm_array

array([[1, 2],
       [3, 4]])

In [45]:
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Input
x = torch.tensor([3.0])

In [46]:
# Forward pass
y_pred = w * x + b
print(f"y_pred: {y_pred.item()}")

y_pred: 7.0


In [47]:
# Loss 

y_true = torch.tensor([10.0])
loss = (y_pred - y_true) ** 2
print(f"Loss: {loss.item()}")

Loss: 9.0


In [48]:
print(f"Avant backward: w.grandd = {w.grad}")

Avant backward: w.grandd = None


In [49]:
loss.backward()

In [50]:
print(f"Apres backward: w.grad = {w.grad}") # Gradient calculé!
print(f"Apres backward: b.grad = {b.grad}")

Apres backward: w.grad = tensor([-18.])
Apres backward: b.grad = tensor([-6.])


In [51]:
# Update manuel des poids 
learning_rate = 0.01
with torch.no_grad(): # Désactiver le tracking des gradients pour l'update w = w + alpha * w
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

In [52]:
print(f"Nouveaux poids: w = {w}, b = {b}")

Nouveaux poids: w = tensor([2.1800], requires_grad=True), b = tensor([1.0600], requires_grad=True)


In [53]:
# Réinitialiser les gradient (IMPORTANT!)
w.grad.zero_()
b.grad.zero_()
print(f"Après zero_ : w.grad = {w.grad}, b.grad = {b.grad}")


Après zero_ : w.grad = tensor([0.]), b.grad = tensor([0.])


In [54]:
import torch.nn as nn

perceptron = nn.Linear(1, 1)

# Voir les poids initiaux
print(f"Poids: {perceptron.weight}")
print(f"Biais: {perceptron.bias}")

Poids: Parameter containing:
tensor([[0.6401]], requires_grad=True)
Biais: Parameter containing:
tensor([0.2029], requires_grad=True)


In [55]:
# Forward
x = torch.tensor([[3.0]]) # Shape (batch_size, features)
print(f"input: {x}")
y_pred = perceptron(x)
print(y_pred)

input: tensor([[3.]])
tensor([[2.1231]], grad_fn=<AddmmBackward0>)


In [56]:
# Loss

y_true = torch.tensor([[10.0]])
criterion = nn.MSELoss()
loss = criterion(y_pred, y_true)

In [57]:
loss.backward()

In [58]:
print(f"Gradient du poids: {perceptron.weight.grad}")
print(f"Gradient du biais: {perceptron.bias.grad}")

Gradient du poids: tensor([[-47.2615]])
Gradient du biais: tensor([-15.7538])


In [61]:
import torch.optim as optim
optimizer = optim.SGD(perceptron.parameters(), lr=0.01)

In [62]:
optimizer.step()

In [63]:
print(f"Nouveau poids: {perceptron.weight} {perceptron.bias}")

Nouveau poids: Parameter containing:
tensor([[1.1127]], requires_grad=True) Parameter containing:
tensor([0.3604], requires_grad=True)


In [64]:
optimizer.zero_grad()

In [ ]:
print(f"Gradient du poids: {perceptron.weight.grad}") 
print(f"Gradient du biais: {perceptron.bias.grad}")

In [65]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [66]:
model = SimpleNet()

In [67]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [68]:
x = torch.tensor([1.0, 2.0])
y = torch.tensor([5.0])
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y)

loss.backward()

In [83]:
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import matplotlib.pyplot as plt

In [91]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [92]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [87]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [73]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits


In [75]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [76]:
x = torch.rand(1,28,28, device = device)
logits =model(x)

In [77]:
logits

tensor([[ 0.0381, -0.0634, -0.1178, -0.1404,  0.0026, -0.1041,  0.0636, -0.0056,
         -0.0093,  0.0309]], grad_fn=<AddmmBackward0>)

In [78]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.1069, 0.0966, 0.0914, 0.0894, 0.1031, 0.0927, 0.1096, 0.1023, 0.1019,
         0.1061]], grad_fn=<SoftmaxBackward0>)

In [80]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([6])

In [98]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [99]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch_idx, batch_value in enumerate(dataloader):
      X, y = batch_value
      X, y = X.to(device), y.to(device)

      # Forward pass
      pred = model(X)
      loss = loss_fn(pred,y)

      # Backward pass
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch_idx % 100 == 0:
          loss, current = loss.item(), (batch_idx+1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [100]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [101]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------


c:\Users\rapha\OneDrive\Bureau\machine-learning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss: 0.922564  [   64/60000]
loss: 0.290820  [ 6464/60000]
loss: 0.169587  [12864/60000]
loss: 0.239409  [19264/60000]
loss: 0.207386  [25664/60000]
loss: 0.047083  [32064/60000]
loss: 0.379361  [38464/60000]
loss: 0.247332  [44864/60000]
loss: 0.093714  [51264/60000]
loss: 0.254730  [57664/60000]
Test Error: 
 Accuracy: 96.8%, Avg loss: 0.104545 

Epoch 2
-------------------------------
loss: 0.067760  [   64/60000]
loss: 0.146889  [ 6464/60000]
loss: 0.079255  [12864/60000]
loss: 0.048304  [19264/60000]
loss: 0.060914  [25664/60000]
loss: 0.161801  [32064/60000]
loss: 0.064268  [38464/60000]
loss: 0.087149  [44864/60000]
loss: 0.023592  [51264/60000]
loss: 0.045757  [57664/60000]
Test Error: 
 Accuracy: 97.4%, Avg loss: 0.079942 

Epoch 3
-------------------------------
loss: 0.019405  [   64/60000]
loss: 0.059604  [ 6464/60000]
loss: 0.027852  [12864/60000]
loss: 0.239214  [19264/60000]
loss: 0.042573  [25664/60000]
loss: 0.031741  [32064/60000]
loss: 0.021826  [38464/60000]
loss: 

In [102]:
example_input = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
onnx_program = torch.onnx.export(model, example_input, dynamo=True)
onnx_program.save("model.onnx")

[torch.onnx] Obtain model graph for `NeuralNetwork([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NeuralNetwork([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
